<a href="https://colab.research.google.com/github/SumeetChougule/PM-HR/blob/main/notebooks/CNN_trial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## git clone

In [3]:
!git clone https://github.com/DifferentiableUniverseInitiative/flowpm.git

Cloning into 'flowpm'...
remote: Enumerating objects: 3470, done.
remote: Counting objects: 100% (1315/1315), done.
remote: Compressing objects: 100% (463/463), done.
remote: Total 3470 (delta 855), reused 1230 (delta 832), pack-reused 2155
Receiving objects: 100% (3470/3470), 65.55 MiB | 14.81 MiB/s, done.
Resolving deltas: 100% (2284/2284), done.


In [4]:
!pip install git+https://github.com/DifferentiableUniverseInitiative/flowpm.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/DifferentiableUniverseInitiative/flowpm.git to /tmp/pip-req-build-prxmwl4j
  Running command git clone -q https://github.com/DifferentiableUniverseInitiative/flowpm.git /tmp/pip-req-build-prxmwl4j
     |████████████████████████████████| 385 kB 26.2 MB/s 
     |████████████████████████████████| 221 kB 75.5 MB/s 
     |████████████████████████████████| 1.1 MB 57.7 MB/s 
  Created wheel for flowpm: filename=flowpm-0.1.2.dev252+g8f9415b-py3-none-any.whl size=45196 sha256=0a2b8eb382782cae40a2f2a48c1eb5512f813bdfa5f7cac3a3a1521a456ed812
  Stored in directory: /tmp/pip-ephem-wheel-cache-hpjlgy4c/wheels/62/64/5f/3dd430bb0fd93140aa11dda1ffbb7a9c3cffb9a3743c997423
  Created wheel for bigfile: filename=bigfile-0.1.51-cp37-cp37m-linux_x86_64.whl size=568150 sha256=f91a4aea4e07438abceee4cb17975da7f83e7b59e6f3135dc27f6c3b612a0aa2
  Stored in directory: /root/.cache/pip/whee

## Modules

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras.layers as tfl
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
from tensorflow.keras import datasets, layers, models, losses

import tensorflow_probability as tfp
tfd = tfp.distributions

In [2]:
import flowpm
from astropy.cosmology import Planck15
from flowpm import linear_field, lpt_init, nbody, cic_paint, cic_readout
from flowpm.utils import r2c3d, c2r3d

from scipy.interpolate import InterpolatedUnivariateSpline as iuspline


# Input data from PM






In [3]:
bs, nc = 100, 32
nsteps = 5
a0, af, nsteps = 0.1, 1.0,  nsteps
stages = np.linspace(a0, af, nsteps, endpoint=True)
donbody = False
dnoise = 1. #0.1

klin, plin = np.loadtxt('flowpm/flowpm/data/Planck15_a1p00.txt').T
ipklin = iuspline(klin, plin)

In [4]:
@tf.function
def pm(linear):
    print("PM graph")
    cosmo = flowpm.cosmology.Planck15()
    state = lpt_init(cosmo, linear, a=a0, order=2)
    final_state = nbody(cosmo, state,  stages, nc)
    tfinal_field = cic_paint(tf.zeros_like(linear), final_state[0])
    return final_state, tfinal_field

In [5]:
ic = linear_field(nc, bs, ipklin, name='pm').numpy()
state, fin = pm(tf.constant(ic))
data = fin + np.random.normal(0, dnoise, nc**3).reshape(fin.shape)

data = data.numpy().astype(np.float32)

PM graph


In [14]:
data.shape

(1, 32, 32, 32)

In [9]:
ip_cnn = tf.expand_dims(data,-1)
ip = ip_cnn.shape
ip[1:]

TensorShape([32, 32, 32, 1])

CNN $→$ CIC_readout $→$ Flatten $→ MLP $

In [14]:
def cnn_model(input_shape):
    model = tf.keras.Sequential([
        tfl.InputLayer(input_shape= input_shape),
        tfl.Conv3D(filters = 32, kernel_size= 3, strides=(2,2,2), padding='same', activation= 'relu',data_format='channels_last'),
        tfl.MaxPooling3D(pool_size=(2,2,2), strides = None, padding ='valid',data_format='channels_last'),
        tfl.Conv3D(filters = 64, kernel_size= 3, strides=(2,2,2), padding='same', activation= 'relu',data_format='channels_last'),
        tfl.MaxPooling3D(pool_size=(3,3,3), strides = None, padding ='valid',data_format='channels_last')
        ])
    return model

cnn = cnn_model(ip[1:])

In [15]:
cnn_op=cnn(data).numpy().astype(np.float32)

In [16]:
cnn_op.shape

(1, 1, 1, 1, 64)

In [17]:
cnn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_6 (Conv3D)           (None, 16, 16, 16, 32)    896       
                                                                 
 max_pooling3d_6 (MaxPooling  (None, 8, 8, 8, 32)      0         
 3D)                                                             
                                                                 
 conv3d_7 (Conv3D)           (None, 4, 4, 4, 64)       55360     
                                                                 
 max_pooling3d_7 (MaxPooling  (None, 1, 1, 1, 64)      0         
 3D)                                                             
                                                                 
Total params: 56,256
Trainable params: 56,256
Non-trainable params: 0
_________________________________________________________________


Interpolation of CNN output

In [19]:
state[0].shape

TensorShape([1, 32768, 3])

In [26]:
mesh = tf.zeros([1, nc, nc, nc])
p_pos = cic_readout( mesh , state[0])
MLP_ip = tfl.Flatten()(p_pos)
MLP_ip.shape

TensorShape([1, 32768])

In [28]:
def MLP():
  model = tf.keras.Sequential([
      tfl.InputLayer(input_shape = MLP_ip.shape),
      tfl.Dense(64, activation = 'relu'),
      tfl.Dense(32, activation = 'relu'),
# how many layers for the output?      
      tfl.Dense( 2, activation = 'linear')
  ])
  return model 

f_op = MLP()

In [29]:
f_op.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 64)             2097216   
                                                                 
 dense_1 (Dense)             (None, 1, 32)             2080      
                                                                 
 dense_2 (Dense)             (None, 1, 2)              66        
                                                                 
Total params: 2,099,362
Trainable params: 2,099,362
Non-trainable params: 0
_________________________________________________________________
